# Лекция 7. Матричные функции, часть 1

## Простейшая матричная функция: матричный полином

Матричный полином имеет очень простой вид

$$ P(A) = \sum_{k=0}^n c_k A^k. $$

[Теорема Гамильтона-Кэли](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%93%D0%B0%D0%BC%D0%B8%D0%BB%D1%8C%D1%82%D0%BE%D0%BD%D0%B0_%E2%80%94_%D0%9A%D1%8D%D0%BB%D0%B8) утверждает, что $F(A) = 0$ где $F(\lambda) = \det(A - \lambda I)$.

## Матричный полином как способ построения любой матричной фугнкции

Можно определить функцию от матрицы с помощью ряда Тейлора:  

$$ f(A) = \sum_{k=0}^{\infty} c_k A^k. $$

Сходимость означает как сходимость в некоторой **матричной норме**.  

Примером такого ряда является ряд Неймана

$$ (I - F)^{-1} = \sum_{k=0}^{\infty} F^k, $$

который определён для $\rho(F) < 1$.

## Ряд для матричной экспоненты

Наиболее известной матричной функцией является **матричная экспонента**. В скалярном случае ряд выглядит следующим образом  

$$ e^x = 1 + x + \frac{x^2}{2} + \frac{x^3}{6} + \ldots = \sum_{k=0}^{\infty} \frac{x^k}{k!}, $$

и он напрямую обобщается на матричный случай:  

$$ e^A = \sum_{k=0}^{\infty} \frac{A^k}{k!}. $$

Этот ряд всегда сходится, так как выполнено следующее равенство

$$\sum_{k=0}^{\infty} \frac{\Vert A \Vert^k}{k!} = e^{\Vert A \Vert}.$$

## Почему матричная экспонента важна?

Огромное количество прикладных задач сводится к системе обыкновенных дифференциальных уравнений вида

$$ \frac{dy}{dt} = Ay, \quad y(0) = y_0. $$


## Обыкновенные дифференциальные уравнения и матричная экспонента

- Дано уравнение

$$\frac{dy}{dt} = Ay, \quad y(0) = y_0.$$

- Формально решение задаётся выражением $y(t) = e^{At} y_0$, поэтому если известна $e^{At}$ (или мы можем быстро умножить матричную экспоненту на вектор), то решение можно получить гораздо быстрее по сравнению с методами, основанными на шагах по времени  
- Действительно,

$$\frac{d}{dt} e^{At} = \frac{d}{dt} \sum_{k=0}^{\infty} \frac{t^k A^k}{k!} = \sum_{k=1}^{\infty} \frac{t^{k-1} A^{k}}{(k-1)!}  = A e^{At}.$$

## Матричная экспонента и шаги по времени

Матричная экспонента может быть гораздо лучше, чем решение с помощью, например, схемы Эйлера:

$$\frac{dy}{dt} \approx \frac{y_{k+1} - y_k}{\tau} = A y_k, \quad y_{k+1} = y_k + \tau A y_k,$$

если мы знаем как вычислить произведение матричной экспоненты на вектор, используя только произведения матрицы $A$ на вектор.

Для плотных матриц матричная экспонента даёт **точный** ответ для ОДУ в любой момент времени $t$ по сравнению с приближённым решением, полученным из схемы Эйлера или схожих подходов.

## Как вычислять матричные функции, включая матричную экспоненту?

- Существует очень много методов даже для матричной экспоненты!

- См. статью [C. Van Loan, C. Moler, Nineteen Dubious Ways to Compute the Exponential of a Matrix, Twenty-Five Years Later](http://www.cs.cornell.edu/cv/researchpdf/19ways+.pdf)

- Самый простой метод – это диагонализация матрицы:  

$$ A = S \Lambda S^{-1}, $$

где столбцы $S$ – собственные векторы матрицы $A$, тогда

$$ F(A) = S F(\Lambda) S^{-1}. $$

**Проблема: диагонализация неустойчива!** (и не любая матрица диагонализуема)

Далее короткое демо

In [1]:
import numpy as np
eps = 1e-4
p = 4
a = np.eye(p)
for i in range(p-1):
    a[i, i+1] = 1
    
a[p-1, 2] = eps
print(a)
val, vec = np.linalg.eig(a)
#print a
print(np.linalg.norm(a - vec.dot(val[:, np.newaxis] * np.linalg.inv(vec))))
#print 'S * D * S^{-1}:' 
print(vec.dot(val[:, np.newaxis] * np.linalg.inv(vec)))

[[1.e+00 1.e+00 0.e+00 0.e+00]
 [0.e+00 1.e+00 1.e+00 0.e+00]
 [0.e+00 0.e+00 1.e+00 1.e+00]
 [0.e+00 0.e+00 1.e-04 1.e+00]]
10000.000049999939
[[ 1.00000000e+00  0.00000000e+00 -7.59220793e-13  1.00000000e+04]
 [ 0.00000000e+00  1.00000000e+00  1.00000000e+00 -4.19097369e-13]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e-04  1.00000000e+00]]


Сейчас мы вычислим матричную экспоненту с помощью диагонализации от **возмущённой Жордановой клетки**.

In [2]:
import numpy as np
eps = 1e-16
p = 5
a = np.eye(p)
for i in range(p-1):
    a[i, i+1] = 1
    
a[p-1, 0] = eps
a = np.array(a)
val, vec = np.linalg.eig(a)
print(np.linalg.norm(a - vec.dot(np.diag(val)).dot(np.linalg.inv(vec))))

fun = lambda x: np.exp(x)

#Using diagonalization
fun_diag = vec.dot(np.diag(fun(val))).dot(np.linalg.inv(vec))


#Using Schur
import scipy.linalg
fun_m = scipy.linalg.expm(a)
print('Difference = {}'.format(np.linalg.norm(fun_m - fun_diag)))

2.0
Difference = 5.959978842992802


## Как работает функция ```funm```?

- Матричная экспонента – это особая функция, и для её вычисления существуют специальные методы.  

- Для произвольной функции $F$, есть замечательный **алгоритм Шура-Парлетта**, который основан для **теореме Шура**

## Алгоритм Шура-Парлетта

- Для данной матрицы $A$ мы хотим вычислить $F(A)$, и можем вычислить $F$ только в **скалярных точках**

- Сначала сведём матрицу $A$ к **треугольной форме**  

$$ A = U T U^*. $$

- Поэтому  $F(A)=U F(T) U^*$

- Нам осталось вычислить функцию от треугольной матрицы.

## Вычисление функции от треугольных матриц

- Мы знаем значения на диагоналях

$$ F_{ii} = F(T_{ii}), $$

также мы знаем, что

$$ F T = T F, $$

то есть значение матричной функции **коммутирует** с самой матрицей. 

- Матричная функция от треугольной матрицы есть треугольная матрица.
- Используя известные значения на главной диагонали и свойство коммутативности, мы получим последовательно остальные диагонали:

$$f_{ij} = t_{ij} \frac{f_{ii} - f_{jj}}{t_{ii} - t_{jj}} + \sum_{k=i+1}^{j-1} \frac{f_{ik} t_{kj} - t_{ki}f_{kj}}{t_{ii} - t_{jj}}.$$

## Матричные функции: ещё одно определение

- Одним из способов определение матричной функции $f(A)$ является использование **канонической формы Жордана**.

- Более элегантный способ определить матричные функции – это использовать **интегральное представление Коши:**

$$
    f(A) = \frac{1}{2\pi i} \int_{\Gamma} f(z) (zI - A)^{-1} dz,
$$

где $f(z)$ аналитическая функция на границе и внутри замкнутого контура $\Gamma$, который покрывает спектр матрицы $A$.

- Определение можно обобщить на случай **операторов**

## Важные матричные функции

- Матричная экспонента используется для решения ОДУ $\frac{dy}{dt} = Ay$ в явном виде $y = e^{At}y_0.$
- $\cos(A), \sin(A)$ используются для решения волнового уравнения $\frac{d^2 y}{dt^2} + Ay = 0.$
- Функция знака, $\mathrm{sign}(A)$, используется для вычисления **спектральных проекций.**
- Обратный квадратный корень из матрицы $A^{-1/2}$ необходим в различных задачах, например, для генерирования сэмплов из нормального распределения